In [3]:
import pandas as pd
import numpy as np
from collections import defaultdict
from gym import Env, spaces
import random
import itertools
import pickle

In [4]:
potential_demand = np.arange(50, 151)  # Possible demand values from 50 to 150
stock_list = np.arange(50, 151)       # Possible stock values from 50 to 150
expected_sales = {}                   # Dictionary to store expected sales


for stock in stock_list:
    cumulative_expected_sales = 0
    for demand_realization in potential_demand:
        sales = min(demand_realization, stock)
        cumulative_expected_sales += sales

    # Calculate the average sales and store it in the dictionary
    expected_sales[stock] = cumulative_expected_sales / len(potential_demand)

In [14]:
payoff_matrix = []

w_range = range(3, 13)
action_range = range(5)

for w in w_range:
    for action in action_range:

        
        optimal_stock = 100 * ((12 - w) / 12) + 50

        if action == 0:
            Q = optimal_stock * 0.8
        elif action == 1:
            Q = optimal_stock * 0.9
        elif action == 2:
            Q = optimal_stock
        elif action == 3:
            Q = optimal_stock * 1.1
        elif action == 4:
            Q = optimal_stock * 1.2
        else:
            Q = optimal_stock

        Q = int(round(Q))
        Q = max(0, min(Q, 150))

        sales = expected_sales.get(Q, 0)
        leftovers = Q - sales

        c = 3
        p = 12
        
        retailer_profit = p * sales - w * Q
        manufacturer_profit = (w - c) * Q

        payoff_matrix.append({
            "w": w,
            "action": action,
            "retailer_profit": retailer_profit,
            "manufacturer_profit": manufacturer_profit
        })

In [15]:
# To store the results
results = []

# Manufacturer decides w first
for w in w_range:
    best_action = None
    best_retailer_profit = float('-inf')
    Q_best = 0

    # Retailer decides action based on the given w
    for action in action_range:
        optimal_stock = 100 * ((12 - w) / 12) + 50
        if action == 0:
            Q = optimal_stock * 0.8
        elif action == 1:
            Q = optimal_stock * 0.9
        elif action == 2:
            Q = optimal_stock
        elif action == 3:
            Q = optimal_stock * 1.1
        elif action == 4:
            Q = optimal_stock * 1.2

        Q = int(round(Q))
        Q = max(0, min(Q, 150))

        sales = expected_sales.get(Q, 0)
        retailer_profit = 12 * sales - w * Q

        if retailer_profit > best_retailer_profit:
            best_retailer_profit = retailer_profit
            best_action = action
            Q_best = Q

    # Manufacturer calculates their profit based on retailer's best action
    manufacturer_profit = (w - 3) * Q_best
    results.append({
        "w": w,
        "action": best_action,
        "retailer_profit": best_retailer_profit,
        "manufacturer_profit": manufacturer_profit
    })

# Find the optimal (w, action) pair that maximizes manufacturer profit
optimal_result = max(results, key=lambda x: x["manufacturer_profit"])

# Output the result
print(f"Optimal Manufacturer Price (w): {optimal_result['w']}")
print(f"Retailer's Optimal Action: {optimal_result['action']}")
print(f"Retailer Profit: {optimal_result['retailer_profit']}")
print(f"Manufacturer Profit: {optimal_result['manufacturer_profit']}")

Optimal Manufacturer Price (w): 10
Retailer's Optimal Action: 2
Retailer Profit: 115.82178217821786
Manufacturer Profit: 469


In [30]:
# Convert to DataFrame for easy manipulation
df = pd.DataFrame(payoff_matrix)

# Pivot tables for each type of profit
manufacturer_profit_table = df.pivot(index="w", columns="action", values="manufacturer_profit")
retailer_profit_table = df.pivot(index="w", columns="action", values="retailer_profit")

# Write to Excel
with pd.ExcelWriter("payoff_matrix.xlsx") as writer:
    manufacturer_profit_table.to_excel(writer, sheet_name="Manufacturer Profit")
    retailer_profit_table.to_excel(writer, sheet_name="Retailer Profit")

print("Payoff matrix saved to payoff_matrix.xlsx")

Payoff matrix saved to payoff_matrix.xlsx
